In [1]:
%load_ext autoreload
%autoreload 2

# UVIS G280 Simulator compared to published data
This notebook runs the simulator and compares the output to the data published in Wakeford et al. (2020, AJ)
We then plot the results to visually show the application of the simulator. 



In [2]:
import UVIS_precision_calculator as upc
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.gridspec as grid
import seaborn as sns

sns.set(style="ticks", palette="colorblind", color_codes=True, context="talk")

In [3]:
%matplotlib qt5

Run the UVIS simulator on the file you want. We will follow the example in the UVIS precision calculator

#### INPUT

`data_folder = '..Data/'`
	The place where the ETC output file located
	The ETC file needs to contain the following information in the correct structure. 
	This file can be self generated if also in this structure.
	(wavelength (A), readnoise, dark_counts, sky_counts, target_counts, total_counts)
	Columns 0 and 4 are used in this calculation, please maintain these column 
	positions for self input files
	Wavelength is input from this file in Angstroms 

`file_name = ['H41_ETC']`
	This can be an array of filenames 
	make sure it is always in square brackets

`exposure_time = [190]` 
	This is an array of exposuretimes corresponding to the 
	filenames listed in file_name
	make sure it is always in square brackets

`orbits_in_transit = [2]` 
	This is an array of the number of HST orbits inside a single
	transit (not the total observation) corresponding to the
	filenames listed in file_name
	make sure it is always in square brackets

`no_of_transits = [2]` 
	This is an array of the number of transits of each targets 
	in your program corresponding to the filenames listed in
	file_name make sure it is always in square brackets

`wl = [2000,8000]`
	default value spans the maximum range of the grism. This will 
	calculate the broadband signal that will be the first line of
	the produced file. 

`startw = 2000`
	This is the starting range for your spectral bins.
	Default is at minimum = 2000 angstroms. 
	Where this might be changed is for small stars with little 
	flux in the short wavelengths. Or if simulating the -1 order

`endw = 8000`
	This is the end of the spectral binning range
	Default set to the maximum, 8000 angstroms

`binlen = 100`
	This is in angstroms and specifies the size of the 
	spectral bins in wavlength space. 
	Minimum reccomended = 60 angstorms (Wakeford et al 2020 = 100)

`out_name = 'string'`
	This is what you want the output specified as in addition 
	to the input name.

#### OUTPUT

data file for each input planet file:
output name = {data_folder}{file_name}.UVIS_{out_name}_sim.txt'
	contains the broadband {wl} and spectroscopic precisions ({binlen} bins) 
	wavelength (nm), bin size (nm), precision per transit (ppm), precision per target (ppm)


In [4]:
data_folder = '' 

file_name = ['H41_ETC'] 

exposure_time = [190] 

orbits_in_transit = [2] # floor(Transit duration / HST period)

no_of_transits = [2] 

wl = [2000,8000]

startw = 2000

endw = 8000

binlen = 100

out_name = '10nm_transmission'

upc.UVIS_simulation(data_folder,file_name,exposure_time,orbits_in_transit,no_of_transits,wl,startw,endw,binlen,out_name)


H41_ETC
exp time =  190
exp in transit =  23.0
exp per orbit =  12.0
precision/transit =  37.94149373653571
(60, 4)
0 [2000, 8000] 100 ['H41_ETC'] [190] [2] [2]
Data has been saved as:  H41_ETC.UVIS_10nm_transmission_sim.csv


In [5]:
h41_sim = pd.read_csv(
    "H41_ETC.UVIS_10nm_transmission_sim.csv",
    header = 3,
    escapechar = '#',
    #engine = "python",
)
h41_sim.rename(columns=str.strip, inplace=True)
h41_sim

,Wavelength (nm),bin size (nm),transit/eclipse precision (ppm),program precision (ppm)
0,500,300.00000,37.94149,26.82869
1,205,5.08475,939.67081,664.44760
2,215,5.08475,478.54751,338.38419
3,225,5.08475,443.86274,313.85835
4,236,5.08475,531.81725,376.05158
5,246,5.08475,412.64102,291.78127
6,256,5.08475,510.34479,360.86826
7,266,5.08475,357.96743,253.12120
8,276,5.08475,459.27839,324.75887
9,286,5.08475,310.36939,219.46430


We will now load in that file and the measured data presented in Wakeford et al. (2020) for comparison

In [6]:
#h41_sim = np.loadtxt('H41_ETC.UVIS_10nm_transmission_sim.txt', dtype=float)
wav_sim = h41_sim["Wavelength (nm)"]
waverr_sim = h41_sim["bin size (nm)"]
precision1_sim = h41_sim["transit/eclipse precision (ppm)"]
precision2_sim = h41_sim["program precision (ppm)"]

h41_data = np.loadtxt('H41_data_Wakeford2020.txt', dtype=float)
wav_data = h41_data[:,0]
h41_depth_marg = h41_data[:,1]
h41_deptherr_marg = h41_data[:,2]*1e4 #convert from % to ppm
h41_depth_jitter = h41_data[:,3]
h41_deptherr_jitter = h41_data[:,4]*1e4 #convert from % to ppm

In the next cell we will plot the two datasets from Wakeford et al. (2020) against the simulated data from this calculator. 

In [7]:
mpl.rc('font', size=20)
mpl.rcParams['figure.figsize'] = [8,6]
plt.figure()
position = grid.GridSpec(3,1, wspace=None)

plt.plot(wav_sim[1:], precision2_sim[1:], color='r', label='UVIS G280 simulation with HST ETC', drawstyle='steps-mid', zorder=6)
plt.plot(wav_data, h41_deptherr_marg, color='b', label='Marginalization method G280 (Wakeford+2020)', drawstyle='steps-mid')
plt.plot(wav_data, h41_deptherr_jitter, color='k', label='Jitter method (Wakeford+2020)', drawstyle='steps-mid')
plt.hlines(precision2_sim[0],100, 1000, color='r', linestyle='--')
plt.hlines(28,100, 1000, color='b', linestyle='--')
plt.hlines(33,100, 1000, color='k', linestyle='--')

plt.xlim(180,820)
plt.ylim(0,1000)

plt.xlabel('Wavelength (nm)')
plt.ylabel('Precision (ppm)')
plt.legend(loc='upper right', fontsize='xx-small')

plt.tight_layout()


This shows that the simulator is a good approximation of the results presented in Wakeford et al. (2020) with some minor differences at the short and long wavelengths. Overall we expect this simulator to be used for proposal purposes only and is an approximation of the expected precision. 